In [2]:
from brax import envs
from prefacc.training.agents.prefppo import train as prefppo

In [3]:
env = envs.get_environment(env_name="ant", backend="positional")

In [4]:
make_inference_fn, params, _ = prefppo.train(
  env,
  num_timesteps=50_000_000, 
  num_evals=10, 
  reward_scaling=10, 
  episode_length=1000, 
  normalize_observations=True, 
  action_repeat=1, 
  unroll_length=5, 
  num_minibatches=32, 
  num_updates_per_batch=4, 
  discounting=0.97, 
  learning_rate=3e-4, 
  entropy_cost=1e-2, 
  num_envs=4096, 
  batch_size=2048, 
  seed=1,
  min_replay_size=2048,
  num_prefs=1400)

print("Training complete")

Training complete


In [5]:
from brax.io import model
from brax.io import json
from brax.io import html

In [6]:
model.save_params('/tmp/params', params)

In [7]:
params = model.load_params('/tmp/params')
inference_fn = make_inference_fn(params)

In [8]:
env = envs.create(env_name="ant", backend="positional")

In [9]:
import jax

In [10]:
jit_env_reset = jax.jit(env.reset)
jit_env_step = jax.jit(env.step)
jit_inference_fn = jax.jit(inference_fn)

In [11]:
rollout = []
rng = jax.random.PRNGKey(seed=1)
state = jit_env_reset(rng=rng)
for _ in range(1000):
  rollout.append(state.pipeline_state)
  act_rng, rng = jax.random.split(rng)
  act, _ = jit_inference_fn(state.obs, act_rng)
  state = jit_env_step(state, act)

In [12]:
from IPython.display import HTML, clear_output

In [18]:
HTML(html.render(env.sys.tree_replace({'opt.timestep': env.dt}), rollout))